In [1]:
!pip install -q -U keras-nlp
!pip install -q keras --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.4/548.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.9 MB/s eta 0:00:00


In [2]:
import os

os.environ["KERAS_BACKEND"] = "jax"

os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

In [3]:
try:
    import keras
    import keras_nlp
except ImportError:
    print("Keras not installed")


In [4]:
import pandas as pd

df = pd.read_csv("train.csv")
df = df.dropna().reset_index(drop=True)
df = df[:1000]

print(f"Shape after cleaning: {df.shape}")
print(df.head())

Shape after cleaning: (1000, 3)
                                         instruction  \
0  Explain why the following fraction is equivale...   
1  Evaluate this sentence for spelling and gramma...   
2  Classify the following into animals, plants, a...   
3               Refactor this code and add comments.   
4  Based on the information provided, rewrite the...   

                                              output  \
0  The fraction 4/16 is equivalent to 1/4 because...   
1  There are two spelling errors in the sentence....   
2  Animals: Elephant\nPlants: Oak tree\nMinerals:...   
3  ```\n# This is a function that calculates the ...   
4  She will play the piano beautifully for hours ...   

                                               input  
0                                               4/16  
1       He finnished his meal and left the resturant  
2                     Oak tree, copper ore, elephant  
3  def factorialize(num):\n  factorial = 1\n  for...  
4  She played the p

In [5]:
df.columns=['instruction','response','context']
df.head()

,instruction,response,context
0,Explain why the following fraction is equivale...,The fraction 4/16 is equivalent to 1/4 because...,4/16
1,Evaluate this sentence for spelling and gramma...,There are two spelling errors in the sentence....,He finnished his meal and left the resturant
2,"Classify the following into animals, plants, a...",Animals: Elephant\nPlants: Oak tree\nMinerals:...,"Oak tree, copper ore, elephant"
3,Refactor this code and add comments.,```\n# This is a function that calculates the ...,def factorialize(num):\n factorial = 1\n for...
4,"Based on the information provided, rewrite the...",She will play the piano beautifully for hours ...,She played the piano beautifully for hours and...


In [6]:
data = []

for i in range(1000):
    data.append("Instruction:\n"+df['instruction'][i]+"\n\nResponse:\n"+df['response'][i])

data = data[:1000]
data[:5]

['Instruction:\nExplain why the following fraction is equivalent to 1/4\n\nResponse:\nThe fraction 4/16 is equivalent to 1/4 because both fractions represent the same value. A fraction can be simplified by dividing both the numerator and the denominator by a common factor. In this case, 4 is a common factor of both the numerator and the denominator of 4/16. When we divide both by 4, we get 4/4 = 1 and 16/4 = 4, so the simplified fraction is 1/4. Alternatively, we can think of this in terms of multiplication. For example, if we multiply the numerator and denominator of the fraction 1/4 by 4, we get (1x4)/(4x4), or 4/16. Since both fractions can be derived from the other through multiplication or division by the same number, they represent the same value and are equivalent.',
 'Instruction:\nEvaluate this sentence for spelling and grammar mistakes\n\nResponse:\nThere are two spelling errors in the sentence. The corrected sentence should be: "He finished his meal and left the restaurant."

In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

ref                                                             title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
lainguyn123/student-performance-factors                         Student Performance Factors                          94KB  2024-09-02 10:53:57          29012        517  1.0              
abdulszz/spotify-most-streamed-songs                            Spotify Most Streamed Songs                          60KB  2024-09-07 18:23:14          11117        146  1.0              
valakhorasani/mobile-device-usage-and-user-behavior-dataset     Mobile Device Usage and User Behavior Dataset        11KB  2024-09-28 20:21:12           3728         71  1.0              
zafarali27/car-price-prediction                             

In [8]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [9]:
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

In [10]:
prompt = template.format(
    instruction="Find the area of a circle given its radius.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=128))

Instruction:
Find the area of a circle given its radius.

Response:
The area of a circle is πr<sup>2</sup>.

Instruction:
Find the area of a circle given its radius.

Response:
The area of a circle is πr<sup>2</sup>.

Instruction:
Find the area of a circle given its radius.

Response:
The area of a circle is πr<sup>2</sup>.

Instruction:
Find the area of a circle given its radius.

Response:
The area of a circle is πr<sup>2</sup>.

Instruction:
Find the area of


In [11]:
prompt = template.format(
    instruction="India Tour guide",
    response="",
)
print(gemma_lm.generate(prompt, max_length=128))

Instruction:
India Tour guide

Response:
Dear Sir,
We are very happy to inform you that we have received your request for the tour guide.
We are pleased to inform you that we have received your request for the tour guide.
We are pleased to inform you that we have received your request for the tour guide.
We are pleased to inform you that we have received your request for the tour guide.
We are pleased to inform you that we have received your request for the tour guide.
We are pleased to inform you that we have received your request for the tour guide.
We are pleased to


In [15]:
gemma_lm.backbone.enable_lora(rank=4)

In [16]:
gemma_lm.preprocessor.sequence_length = 256
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
import gc
gc.collect()
gemma_lm.fit(data, epochs=1, batch_size=1)

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 861s 835ms/step - loss: 0.5660 - sparse_categorical_accuracy: 0.5910


In [19]:
prompt = template.format(
    instruction="Find the area of a circle given its radius.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=128))

Instruction:
Find the area of a circle given its radius.

Response:
The area of a circle with a radius of 5 cm is 78.5 cm².


In [20]:
prompt = template.format(
    instruction="India Tour guide.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=128))

Instruction:
India Tour guide.

Response:
Hello,

Thank you for your interest in India. I am a tour guide and I would love to help you plan your trip to India.

India is a vast and diverse country with a rich history, culture, and natural beauty. From the bustling cities of Delhi and Mumbai to the serene beaches of Goa, there is something for everyone in India.

If you are interested in exploring the history and culture of India, I can help you plan a tour that takes you to some of the most important historical sites and cultural landmarks. From the Taj Mahal in Agra to the Red
